# Heat Geodesics en NIFs

Tenemos la PDE 
$$\frac{\partial u}{\partial t} = \nabla \cdot ( P_{\nabla \psi} \nabla u) = \Delta u - (\nabla u \cdot \nabla \psi) \Delta \psi$$

Definimos $u_0(\vec{x}) = e^{-\varepsilon \left || \vec{x} - \vec{p} | \right |^2}$. Entonces,

$$\nabla u_0(\vec{x}) = -2 \varepsilon u_0(\vec{x}) (\vec{x} - \vec{p}) \quad \quad \text{y} \quad \quad \Delta u_0(\vec{x}) = 2\varepsilon \, u_0(\vec{x}) \, (2 \varepsilon \left || \vec{x} - \vec{p} | \right |^2 - 3) $$

### Forward Euler
Este esquema es mas sencillo (pero mas inestable), integramos con respecto al tiempo tomando la siguiente aproximacion:
$$u_t(\vec{x}) = u_0(\vec{x}) + t (\Delta u_0(\vec{x}) - (\nabla u_0 \cdot \nabla \psi) \Delta \psi)$$

De esta forma,
$$ \nabla u_t(\vec{x}) = \nabla $$

### Backward Euler
Integrador implicito:
$$u_t = u_0 + t (\nabla \cdot \, P_{\nabla \psi} \nabla) u_t$$

Donde se entiende por $(\nabla \cdot \, P_{\nabla \psi}) : C^k \rightarrow C^k$ el operador que actua sobre funciones $u$, el cual aplica la proyeccion del gradiente sobre el espacio ortogonal a $\nabla \psi$ y luego la divergencia.

$$t(id - \nabla \cdot \, P_{\nabla \psi} \nabla) u_t = u_0$$

Entonces, $u_t$ es la funcion que minimiza la energia:

$$ \underset{u: \Omega \rightarrow \mathbb{R}}{\text{min}} \quad \quad \int_\Omega \left | t(id - \nabla \cdot \, P_{\nabla \psi} \,\nabla) u_t - u_0 \right | \quad d\vec{x}.$$

Tengo muchas formas de realizar esta optimizacion:
1. CML dada una familia de funciones
2. Discretizacion y FEM ?
3. Usando el mismo SIREN

Hay que tener fe de que estas minimizaciones ganen ventaja frente a FE... ta dificil.

In [1]:
import numpy as np
import torch
from src.model import SIREN
from src.obj import load
import meshplot as mp
import src.diff_operators as dif
from torchdiffeq import odeint

In [2]:
model = SIREN(
        n_in_features= 3,
        n_out_features=1,
        hidden_layer_config=[512]*4,
        w0=30,
        ww=None,
        activation= 'sine'
)
model.load_state_dict( torch.load('results/bunny/experiment/models/model_best.pth', weights_only=True))

device_torch = torch.device(0)
model.to(device_torch)

/home/mfainstein/miniconda3/envs/tdf/lib/python3.10/site-packages/torch/_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


SIREN(
  (net): Sequential(
    (0): Sequential(
      (0): Linear(in_features=3, out_features=512, bias=True)
      (1): SineLayer(w0=30)
    )
    (1): Sequential(
      (0): Linear(in_features=512, out_features=512, bias=True)
      (1): SineLayer(w0=30)
    )
    (2): Sequential(
      (0): Linear(in_features=512, out_features=512, bias=True)
      (1): SineLayer(w0=30)
    )
    (3): Sequential(
      (0): Linear(in_features=512, out_features=512, bias=True)
      (1): SineLayer(w0=30)
    )
    (4): Sequential(
      (0): Linear(in_features=512, out_features=1, bias=True)
    )
  )
)

In [3]:
def grad_desc( model, samples, max_batch=64**2, device=torch.device(0), iterations = 3 ):
    # samples = ( amount_samples, 3 )
    amount_samples = samples.shape[0]

    head = 0
    X = samples.copy()

    for i in range( iterations):
        print(f'Iteration: {i}')
        mean_distance = 0
        while head < amount_samples:
            
            inputs_subset = torch.from_numpy( X[head:min(head + max_batch, amount_samples), :] ).to(device).unsqueeze(0).float()

            x, y = model(inputs_subset).values()

            mean_distance += torch.sum(y).detach().cpu().numpy()

            grad_psi = torch.nn.functional.normalize( dif.gradient(y,x), dim=-1 )

            X[head:min(head + max_batch, amount_samples)] -= (y *  grad_psi).squeeze(0).detach().cpu().numpy()

            head += max_batch

        print(f'Mean distance: { mean_distance / amount_samples }')

    return X

pc, _ = load( 'data/bunny/bunny_pc.obj' )
#pc = pc[np.random.choice( np.arange(10000), 1000),: ]
pc = grad_desc( model, pc, iterations=1 )

Iteration: 0
Mean distance: -9.786325991153717e-05


In [4]:
t = 1
EPSILON = 0.001

u0 = lambda x, p: torch.exp( - torch.sum( (x - p)**2, dim=-1) / (EPSILON ** 2) )[...,None] / ( EPSILON )
grad_u0 = lambda x, p: (-2 / (EPSILON ** 2)) * u0(x, p) * (x - p)

In [5]:
def project( projectees, projectors ):
    # asumiendo ||projectors|| = 1
    return projectees -  torch.sum(projectors * projectees, dim=-1)[...,None] * projectors

def computeX( model, samples, p, u0, grad_u0, max_batch=64**2, device=torch.device(0) ):
    # samples = ( amount_samples, 3 )    
    head = 0
    amount_samples = samples.shape[0]

    #uts = np.zeros( (amount_samples, 1))
    grads_psi = np.zeros( (amount_samples, 3))
    laplacians_psi = np.zeros( (amount_samples, 1))
    #X = np.zeros( (amount_samples, 3))
    #divX = np.zeros( (amount_samples, 1))

    #ps = torch.from_numpy( np.tile(p, (max_batch, 1)) ).to(device_torch)
    i = 0
    while head < amount_samples:
        print(f'Iteration: {i}')
        
        inputs_subset = torch.from_numpy( samples[head:min(head + max_batch, amount_samples), :] ).to(device).unsqueeze(0).float()

        x, y =  model(inputs_subset).values()

        grad_psi = dif.gradient(y,x)

        #ps_ss = ps[:inputs_subset.shape[1],:].unsqueeze(0)

        # Forward Euler
        #proj_grad_u0 = project( grad_u0( inputs_subset, ps_ss ), grad_psi )
        #F_u0 = dif.divergence( proj_grad_u0 , x )
        #u0s = u0( inputs_subset, ps_ss )
        #ut = u0s + t * F_u0

        # RK4: ( out of memory... lo mata hacer el gradiente de u para calcular X.. es un monton)
        #k1 = dif.divergence( project( grad_u0( inputs_subset, ps_ss ), grad_psi ) , x )
        #k2 = dif.divergence( project( dif.gradient(u0s + 0.5 * t * k1 , x), grad_psi ), x )
        #k3 = dif.divergence( project( dif.gradient(u0s + 0.5 * t * k2 , x), grad_psi ), x )
        #k4 = dif.divergence( project( dif.gradient(u0s + t * k3 , x), grad_psi ), x )
        #ut = u0s + (t/6) * (k1 + 2*k2 + 2*k3 + k4)

        # Heuns:
        #k1 = dif.divergence( proj_grad_u0 , x )
        #k2 = dif.divergence( project( dif.gradient(u0s + t * k1 , x), grad_psi ), x )
        #ut = u0s + (t/2) * (k1 + k2)
        
        #uts[head:min(head + max_batch, amount_samples)] = ut.detach().cpu().squeeze(0).numpy()

        #X_ss = -1 * torch.nn.functional.normalize( proj_grad_u0 + project( dif.gradient( (t/2) * (k1 + k2), x ), grad_psi ), dim=-1, eps=1e-30 )
        #X_ss = -1 * torch.nn.functional.normalize( proj_grad_u0 + t * project( dif.gradient( F_u0 , x), grad_psi ), dim=-1 , eps=1e-30 )
        #X_ss =  torch.nn.functional.normalize( proj_grad_u0 , dim=-1 , eps=1e-30 )
        
        #X[head:min(head + max_batch, amount_samples)] = X_ss.detach().cpu().squeeze(0).numpy()
        #divX[head:min(head + max_batch, amount_samples)] = dif.divergence( X_ss, x ).detach().cpu().squeeze(0).numpy()

        laplacians_psi[head:min(head + max_batch, amount_samples)] = dif.divergence( grad_psi, x ).detach().cpu().squeeze(0).numpy()
        grads_psi[head:min(head + max_batch, amount_samples)] = grad_psi.detach().cpu().squeeze(0).numpy()

        head += max_batch
        i += 1

        torch.cuda.empty_cache()

    return grads_psi, laplacians_psi #, uts, X, divX

grads_psi, laplacians_psi = computeX( model, pc, pc[47,:], u0, grad_u0, max_batch=64**2 )

Iteration: 0
Iteration: 1
Iteration: 2


In [9]:
def F( y, x ):
    coords, sdf =  model(x).values()
    grad_psi = dif.gradient(sdf, coords)

    proj_grad_u0 = project( dif.gradient( y, x ), grad_psi )
    return dif.divergence( proj_grad_u0 , x )

N = 10000
X = torch.from_numpy( pc[:N, :] ).to(device_torch).unsqueeze(0).float()
X.requires_grad_()
p = torch.from_numpy( np.tile( pc[0,:] , (N, 1)) ).to(device_torch)
t = torch.Tensor([0,0.00001, 0.0001]).to(device_torch)
t.requires_grad_()

u0, u1, u2 = odeint( lambda t, y: F(y, X),u0(X, p) , t, method='euler' )

tensor([[[[ 1.0000e+03],
          [ 0.0000e+00],
          [ 0.0000e+00],
          ...,
          [ 0.0000e+00],
          [ 0.0000e+00],
          [ 0.0000e+00]]],


        [[[-3.9106e+04],
          [ 0.0000e+00],
          [ 0.0000e+00],
          ...,
          [ 0.0000e+00],
          [ 0.0000e+00],
          [ 0.0000e+00]]],


        [[[ 2.8476e+07],
          [ 0.0000e+00],
          [ 0.0000e+00],
          ...,
          [ 0.0000e+00],
          [ 0.0000e+00],
          [ 0.0000e+00]]]], device='cuda:0', dtype=torch.float64,
       grad_fn=<CopySlices>)

In [ ]:
clip = lambda x : np.clip( x, np.percentile(x, 1), np.percentile(x,99))

plot = mp.plot( pc, c= clip(laplacians_psi), shading={'point_size':0.3}, return_plot=True )
#plot.add_points( pc[30,:][None,...], shading={'point_size':0.3})
plot.add_lines( pc, pc + grads_psi*0.05 )

In [ ]:
import matplotlib.pyplot as plt
plt.hist( clip(laplacians_psi), 30)


Ahora si.... ecuacion de Poisson:

Buscamos $\phi : \partial \Omega \rightarrow \mathbb{R}$ tal que:

$$\nabla_{\partial \Omega} \phi = X$$

Donde $\nabla_{\partial \Omega} = P_{\nabla \psi} \nabla$,   es el gradiente intrinseco de $\partial \Omega$. Esto suele ser un problema mal definido (?). Se reemplaza por el clasico problema de Poisson:

$$\Delta_{\partial \Omega} \phi := \nabla \cdot \nabla_{\partial \Omega} \phi = \nabla \cdot X $$

Entonces la pregunta es como hago para resolver esta ecuacion dado que:
1. No tenemos valores en una grilla para hacer algo estilo diferencias finitas.
2. No tenemos una malla, por lo tanto no podemos hacer diferencias finitas en malla (como hacen en el paper original) ni elementos finitos.

Siento que hay dos opciones principales:
1. Olvidarse de la SDF de fondo, y usar los metodos para nubes de puntos que usan en el paper original.
2. Pasar a un Octree o algo del estilo y resolver como hacen en PSR.
3. *MonteCarlo* ?????
4. Usar una red SIREN o RBF.

RBF es super facil de implementar definiendo el sistema lineal por cada punto de la nube. El tema es que requiere solucion global (que es ventaja y desventaja).

In [ ]:
from scipy.interpolate import RBFInterpolator

N = pc.shape[0]
G = pc @ pc.T
P = -2 * G + np.outer( np.diag(G), np.ones(N)) + np.outer( np.ones(N), np.diag(G))
eps = 100

phis = np.exp( - eps * P )
laplace_Phis = 2 * eps * phis * (2 * eps * P - 3)

print(laplace_Phis.shape)

In [ ]:
W = pc @ grads_psi.T

D = np.outer( np.diag(W), np.ones(N) ) - W.T

A = laplace_Phis #+ ( 2 * eps * phis * D * np.outer( laplacians_psi, np.ones(N)) )

In [ ]:
a= np.linalg.solve( A, divX )#RBFInterpolator(pc, divX,kernel='gaussian', epsilon=eps, smoothing=9)(pc) )

In [ ]:
distances = np.exp( -eps * P ) @ a

In [ ]:
np.min( distances ), np.max(distances)

In [ ]:
mp.plot( pc, c= distances, shading={'colormap':'Reds' ,'point_size':0.3}, )

In [ ]:
np.min( divX) ,np.max(divX), np.mean(divX), np.std(divX)

## Collocation

Vamo de vuelta... tenemos:
$$\frac{\partial u}{\partial t} = \nabla \cdot ( P_{\nabla \psi} \nabla u) = \Delta u - (\nabla u \cdot \nabla \psi) \Delta \psi$$

... despues lo paso en limpio


In [ ]:
N = pc.shape[0]
G = pc @ pc.T
P = -2 * G + np.outer( np.diag(G), np.ones(N)) + np.outer( np.ones(N), np.diag(G))
eps = 10

M = np.exp( - eps * P )
K = 2 * eps * M * (2 * eps * P - 3)

GtX = grads_psi @ pc.T
T = 2 * eps * M * ( GtX - np.outer( np.diag(GtX), np.ones(N) ) )

D_psi = np.diag( laplacians_psi.flatten() )

In [ ]:
M.shape, K.shape, T.shape

In [ ]:
M

Entonces, ahora que tengo las matrices tengo que encontrar $\vec{\alpha}_0$ que satisfaga la condicion de valor inicial.
Creo que en este caso es $\vec{p} = 1$ y todo el resto en $0$... pero en el paper hacen algo raro. porque en realidad deberia ser la funcion delta? nose... rari

In [ ]:
alpha_0,_,_,_ = np.linalg.lstsq( M, np.eye( 1,N, k=47)[0] * 10000)
alpha_0[47]

In [ ]:
p = mp.plot( pc, c= alpha_0, shading={'point_size':0.25}, return_plot=True )
p.add_points( pc[47,:][None,...], shading={'point_size':0.3})

In [ ]:
t = 0.000001
A = M - t * (K - D_psi @ T)

alpha_t,_,_,_ = np.linalg.lstsq( A, M @ alpha_0)

In [ ]:
mp.plot( pc, c= alpha_t, shading={'point_size':0.25} )

mmmm no parece andar... no me gusta

## Backward Euler con punto fijo

$g(u_k) = (h + 1)u_k - h\nabla \cdot P_{\nabla \psi} \nabla u_k - u_{k-1}$

$u_{k+1} = g(u_k)$

